In [ ]:
import requests
import time
!export PATH=../../tools:$PATH
# Replace with your own docker_id
!export DOCKER_ID=...

In [ ]:
configuration = """{
    "384562a7f005fc5fb4e577c5b4c60617": {
        "parallelism": {
            "lowerBound": 1,
            "upperBound": 1
        },
        "resourceProfile": {
            "cpuCores": {
                "name": "CPU",
                "value": 1,
                "zero": false
            },
            "taskHeapMemory": {
                "bytes": 119453752,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "taskOffHeapMemory": {
                "bytes": 0,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "managedMemory": {
                "bytes": 1492501155,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "networkMemory": {
                "bytes": 186562644,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            }
        }
    },
    "cbc357ccb763df2852fee8c4fc7d55f2": {
        "parallelism": {
            "lowerBound": 1,
            "upperBound": 1
        },
        "resourceProfile": {
            "cpuCores": {
                "name": "CPU",
                "value": 1,
                "zero": false
            },
            "taskHeapMemory": {
                "bytes": 119453752,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "taskOffHeapMemory": {
                "bytes": 0,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "managedMemory": {
                "bytes": 1492501155,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "networkMemory": {
                "bytes": 186562644,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            }
        }
    },
    "bf003860ab139437fc03298928868735": {
        "parallelism": {
            "lowerBound": $PARALLELISM,
            "upperBound": $PARALLELISM
        },
        "resourceProfile": {
            "cpuCores": {
                "name": "CPU",
                "value": 1,
                "zero": false
            },
            "taskHeapMemory": {
                "bytes": 119453752,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "taskOffHeapMemory": {
                "bytes": 0,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "managedMemory": {
                "bytes": $MANAGED,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            },
            "networkMemory": {
                "bytes": 186562644,
                "kibiBytes": 0,
                "mebiBytes": 0,
                "gibiBytes": 0,
                "tebiBytes": 0
            }
        }
    }
}"""

In [ ]:
def run_xp():
    jobs = requests.get("http://localhost:8081/jobs").json()

    job_id = [job['id'] for job in jobs['jobs'] if job['status'] == 'RUNNING'][0]
    
    for parallelism in [1, 2, 4, 8]:
        for managed in [128, 256, 518, 1024, 2048]:
            config = configuration.replace("$PARALLELISM", str(parallelism))
            config = config.replace("$MANAGED", str(managed * 1024 * 1024))
            #print(config)
            r = requests.put(f"http://localhost:8081/jobs/{job_id}/justin", data=config)
            time.sleep(600)

'4874d37ebe237dc4d243b9e0eec4673c'

# Read-Only

In [ ]:

!sed -i -e "s/DOCKER_ID/$DOCKER_ID/g" query.yaml; kubectl apply -f read-only/query.yaml
time.sleep(30)
run_xp()
!kubectl delete -f read-only/query.yaml

{}


# Write-Only

In [ ]:

!sed -i -e "s/DOCKER_ID/$DOCKER_ID/g" query.yaml; kubectl apply -f write-only/query.yaml
time.sleep(30)
run_xp()
!kubectl delete -f write-only/query.yaml

# Update

In [ ]:

!sed -i -e "s/DOCKER_ID/$DOCKER_ID/g" query.yaml; kubectl apply -f update/query.yaml
time.sleep(30)
run_xp()
!kubectl delete -f update/query.yaml

# Observing Throughputs

We can use Grafana to access Prometheus metrics and observe the throughput reached for each configuration.
Credentials are: 
- admin
- prom-operator

[http://localhost:3000/explore?orgId=1&left=%5B%22now-3h%22,%22now%22,%22Prometheus%22,%7B%22refId%22:%22A%22,%22instant%22:true,%22range%22:true,%22exemplar%22:true,%22expr%22:%22sum(flink_taskmanager_job_task_operator_numRecordsOutPerSecond%7Boperator_name%3D~%5C%22Source.*%5C%22%7D)%22%7D%5D](http://localhost:3000/explore?orgId=1&left=%5B%22now-3h%22,%22now%22,%22Prometheus%22,%7B%22refId%22:%22A%22,%22instant%22:true,%22range%22:true,%22exemplar%22:true,%22expr%22:%22sum(flink_taskmanager_job_task_operator_numRecordsOutPerSecond%7Boperator_name%3D~%5C%22Source.*%5C%22%7D)%22%7D%5D)